In [ ]:
import pickle

import numpy as np
import pandas as pd 

from importlib import reload

import helper_funcs;reload(helper_funcs);from helper_funcs import *
import make_models2;reload(make_models2);from make_models2 import *

from functools import partial
from sklearn.decomposition import PCA

from matplotlib import pyplot as plt

from datetime import datetime

import shutil
from functools import partial
from tqdm import tqdm
tqdm = partial(tqdm, position=0, leave=True)

In [ ]:
dataFnOut = '../Data/ABIDE-Anat-64iso-S982.npz'
dfFnOut = '../Data/ABIDE_legend_S982.csv'

ABIDE_data = np.load(dataFnOut)['data']
df = pd.read_csv(dfFnOut)

patients = df['DxGroup'].values==1
controls = df['DxGroup'].values==2
abide_asd = ABIDE_data[patients,:,:,:]

arr = np.load('../Data/SFARI-Anat-64iso-S121.npz')
dfs = pd.read_csv('../Data/sfari_legend_S121.csv')

SFARI_data = arr['data']
SFARI_subs = arr['subs']

sfari_subs_td = dfs['family_type'].values=='non-familial-control'
sfari_subs_dupl = dfs['family_type'].values=='16p-duplication'
sfari_subs_del = dfs['family_type'].values=='16p-deletion'

In [ ]:
patients = df['DxGroup'].values==1
controls = df['DxGroup'].values==2

In [ ]:
# LOAD VAE
latent_dim=32;batch_size=32;disentangle=False;gamma=100
encoder, decoder, vae = get_MRI_VAE_3D(input_shape=(64, 64, 64, 1), latent_dim=latent_dim, batch_size=batch_size, disentangle=disentangle, gamma=gamma)

encoder, decoder, vae = get_MRI_VAE_3D(input_shape=(64,64,64,1), 
                                            latent_dim=32, 
                                            batch_size = batch_size, 
                                            disentangle=True,
                                            gamma=100,
                                            kernel_size = 3,
                                            filters = 48,
                                            intermediate_dim = 128,
                                            nlayers = 2,
                                            bias=True)

fn = '../tf_weights/CVAE_weights/VAE_weights'
vae.load_weights(fn)

In [ ]:
%%time 
# LOAD CVAE
latent_dim = 16;batch_size = 32;beta = 1;gamma = 100;disentangle = True
cvae, z_encoder, s_encoder, cvae_decoder = get_MRI_CVAE_3D(latent_dim=latent_dim,beta=beta, disentangle=disentangle, gamma=gamma, bias=True, batch_size = batch_size)
loss = list()    

fn = '../tf_weights/CVAE_weights/CVAE_weights'
cvae.load_weights(fn)

In [ ]:
n_samples = 10

salient_vec_abide = np.array([s_encoder.predict(ABIDE_data[:,:,:,:])[2] for _ in range(n_samples)])
background_vec_abide = np.array([z_encoder.predict(ABIDE_data[:,:,:,:])[2] for _ in range(n_samples)])
vae_vec_abide = np.array([encoder.predict(ABIDE_data[:,:,:,:])[2] for _ in range(n_samples)])

salient_vec_sfari = np.array([s_encoder.predict(SFARI_data[:,:,:,:])[2] for _ in range(n_samples)])
background_vec_sfari = np.array([z_encoder.predict(SFARI_data[:,:,:,:])[2] for _ in range(n_samples)])
vae_vec_sfari = np.array([encoder.predict(SFARI_data[:,:,:,:])[2] for _ in range(n_samples)])

fn = '../Data/latent_vecs10.npz'
np.savez_compressed(fn, 
                    salient_vec_abide=salient_vec_abide,
                    background_vec_abide=background_vec_abide,
                    vae_vec_abide=vae_vec_abide,
                   salient_vec_sfari=salient_vec_sfari,
                   background_vec_sfari=background_vec_sfari,
                   vae_vec_sfari=vae_vec_sfari)

In [ ]:
n_samples = 100

salient_vec_abide = np.array([s_encoder.predict(ABIDE_data[:,:,:,:])[2] for _ in range(n_samples)])
background_vec_abide = np.array([z_encoder.predict(ABIDE_data[:,:,:,:])[2] for _ in range(n_samples)])
vae_vec_abide = np.array([encoder.predict(ABIDE_data[:,:,:,:])[2] for _ in range(n_samples)])

salient_vec_sfari = np.array([s_encoder.predict(SFARI_data[:,:,:,:])[2] for _ in range(n_samples)])
background_vec_sfari = np.array([z_encoder.predict(SFARI_data[:,:,:,:])[2] for _ in range(n_samples)])
vae_vec_sfari = np.array([encoder.predict(SFARI_data[:,:,:,:])[2] for _ in range(n_samples)])

fn = '../Data/latent_vecs100.npz'
np.savez_compressed(fn, 
                    salient_vec_abide=salient_vec_abide,
                    background_vec_abide=background_vec_abide,
                    vae_vec_abide=vae_vec_abide,
                   salient_vec_sfari=salient_vec_sfari,
                   background_vec_sfari=background_vec_sfari,
                   vae_vec_sfari=vae_vec_sfari)